![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [10]:
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "UCI HAR Dataset/features.txt"
filename_labels = "UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "UCI HAR Dataset/train/y_train.txt"

filename_subtest = "UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "UCI HAR Dataset/test/X_test.txt"
filename_ytest = "UCI HAR Dataset/test/y_test.txt"

features_df = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features = features_df['nome_var']

labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train_df = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_train = subject_train_df['subject_id']

X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test_df = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
subject_test = subject_test_df['subject_id']

X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


C:\Users\shinoki\AppData\Local\Temp\ipykernel_7892\339483026.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
C:\Users\shinoki\AppData\Local\Temp\ipykernel_7892\339483026.py:29: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
C:\Users\shinoki\AppData\Local\Temp\ipykernel_7892\339483026.py:35: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [11]:
%%time

tree = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
tree.fit(X_train, y_train.values.ravel())

y_pred_train = tree.predict(X_train)
y_pred_test = tree.predict(X_test)

acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)

print(f"Acurácia (Treino): {acc_train:.4f}")
print(f"Acurácia (Teste) : {acc_test:.4f}")



Acurácia (Treino): 0.9758
Acurácia (Teste) : 0.8799
CPU times: total: 7.59 s
Wall time: 8.92 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [12]:
%%time

pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

tree_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

tree_pca.fit(X_train_pca, y_train.values.ravel())

y_pred_train_pca = tree_pca.predict(X_train_pca)
y_pred_test_pca = tree_pca.predict(X_test_pca)

acc_train_pca = accuracy_score(y_train, y_pred_train_pca)
acc_test_pca = accuracy_score(y_test, y_pred_test_pca)

print(f"Acurácia (Treino com 1 PCA): {acc_train_pca:.4f}")
print(f"Acurácia (Teste com 1 PCA) : {acc_test_pca:.4f}")

Acurácia (Treino com 1 PCA): 0.4997
Acurácia (Teste com 1 PCA) : 0.4571
CPU times: total: 578 ms
Wall time: 575 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [13]:
%%time

componentes = [1, 2, 5, 10, 50]

for n in componentes:
    print(f"\n--- Testando PCA com {n} componente(s) ---")
    
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    tree = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
    
    start = time.time()
    tree.fit(X_train_pca, y_train.values.ravel())  # Treinar aqui!
    elapsed = time.time() - start
    
    y_pred_train = tree.predict(X_train_pca)
    y_pred_test = tree.predict(X_test_pca)
    
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    
    print(f"Tempo de treinamento: {elapsed:.4f} segundos")
    print(f"Acurácia treino: {acc_train:.4f}")
    print(f"Acurácia teste : {acc_test:.4f}")


--- Testando PCA com 1 componente(s) ---
Tempo de treinamento: 0.0924 segundos
Acurácia treino: 0.4997
Acurácia teste : 0.4571

--- Testando PCA com 2 componente(s) ---
Tempo de treinamento: 0.0918 segundos
Acurácia treino: 0.6128
Acurácia teste : 0.5847

--- Testando PCA com 5 componente(s) ---
Tempo de treinamento: 0.0896 segundos
Acurácia treino: 0.8460
Acurácia teste : 0.7886

--- Testando PCA com 10 componente(s) ---
Tempo de treinamento: 0.1791 segundos
Acurácia treino: 0.8928
Acurácia teste : 0.8191

--- Testando PCA com 50 componente(s) ---
Tempo de treinamento: 0.9411 segundos
Acurácia treino: 0.9189
Acurácia teste : 0.8266
CPU times: total: 4.86 s
Wall time: 3.8 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

In [ ]:
# Poucos componentes: menos tempo, mas pior desempenho.

# Mais componentes: melhor desempenho, mas mais tempo.

# Existe um trade-off entre simplificação (menos componentes, menos tempo) e qualidade do modelo (acurácia).